In [206]:
import numpy as np
import pandas as pd
import random
import string
import tensorflow as tf
from tensorflow import keras
import tensorboard
import os
from pathlib import Path
import datetime

from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import train_test_split

print('TensorFlow version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('Tensorboard version:', tensorboard.__version__)



TensorFlow version:  2.0.0
Keras version:  2.2.4-tf
Tensorboard version: 2.0.0


In [185]:
# city names
canada_cities = pd.read_csv('list-of-cities-in-canada-csv.csv',names=['city'])
l = list(pd.get_dummies(canada_cities,drop_first=True).columns)
l.sort()
k = {}
for i in enumerate(l):
    k[i[1].split('_')[1].lower()] = i[0]
    
len(k)
zero_list = list(np.zeros(len(k)))
zero_list[3] = 1.0

In [98]:
df1 = pd.read_csv(data_path / file_csv[0],)
print(df1.head())
x1 = df2.to_numpy()

   age          city   dist gender    make  year     model new_used  \
0   61    Belleville  10726      m    audi  2017    e-tron     used   
1   21  Wood Buffalo   9401      m  toyota  2001     camry     used   
2   83       Kelowna   9427      m   acura  2016       mdx     used   
3   28      St. John  21090      m    ford  2003  explorer     used   
4   56        Sarnia   7099      m    audi  2018    e-tron     used   

         quote  
0   352.599790  
1  1723.855245  
2   221.792349  
3  1723.855245  
4   354.748787  


In [99]:
df1 = pd.read_csv(data_path / file_csv[0],)

In [69]:
df2 = pd.read_csv(data_path / file_csv[1],)
print(df2.head())
x2 = df2.to_numpy()
x2.shape

   age          city   dist gender    make  year     model new_used  \
0   61    Belleville  10726      m    audi  2017    e-tron     used   
1   21  Wood Buffalo   9401      m  toyota  2001     camry     used   
2   83       Kelowna   9427      m   acura  2016       mdx     used   
3   28      St. John  21090      m    ford  2003  explorer     used   
4   56        Sarnia   7099      m    audi  2018    e-tron     used   

        quote  
0  432.629987  
1  175.000958  
2  263.333491  
3  175.000958  
4  433.598257  


(5000, 9)

In [71]:
b = np.dstack((x1,x2))
b.shape

(5000, 9, 2)

In [201]:
data_path = Path('/home/tim/Documents/flask_experiment/data')

file_csv = []
index_count = 0
for file in os.listdir(data_path):
    # open up the .mat file
    if file.endswith(".csv"):
        file_csv.append(file)
        print(file)
        df = pd.read_csv(data_path / file,)
        df = df.drop(['model'],axis=1,inplace=False)
        if index_count == 0:
            y = df['quote'].to_numpy()
            df = df.drop(['quote'],axis=1,inplace=False)
            df = pd.get_dummies(df,drop_first=True)
            X = df.to_numpy()
            index_count += 1
        else:
            a = df['quote'].to_numpy()
            y = np.vstack((a, y))
            df = df.drop(['quote'],axis=1,inplace=False)
            df = pd.get_dummies(df,drop_first=True)
            X = np.dstack((df.to_numpy(), X))
            index_count += 1

y = np.reshape(y,(-1,5))
X = np.reshape(X,(X.shape[0],-1,X.shape[1]))
print(X.shape)
print(file_csv)



company_4.csv
company_2.csv
company_18.csv
company_1.csv
company_11.csv
(5000, 5, 114)


['company_4.csv',
 'company_2.csv',
 'company_18.csv',
 'company_1.csv',
 'company_11.csv']

In [210]:
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=[5, 114]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(1, activation="linear")
])

In [211]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 5, 300)            34500     
_________________________________________________________________
dense_13 (Dense)             (None, 5, 300)            90300     
_________________________________________________________________
dense_14 (Dense)             (None, 5, 300)            90300     
_________________________________________________________________
dense_15 (Dense)             (None, 5, 1)              301       
Total params: 215,401
Trainable params: 215,401
Non-trainable params: 0
_________________________________________________________________


In [212]:
model.compile(loss="mse",
              optimizer="adam",
              metrics=["mse"])

# create a name for the model so that we can track it in tensorboard
log_dir="logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_quote"

# create tensorboard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0,
                                                      update_freq='epoch',profile_batch=0)

In [213]:
history = model.fit(X, y, 
                    epochs=30,
                    verbose=1,
                    callbacks=[tensorboard_callback])

Train on 5000 samples
Epoch 1/30
5000/5000 [==============================] - 1s 169us/sample - loss: 165519.9183 - mse: 165519.8906
Epoch 2/30
5000/5000 [==============================] - 0s 80us/sample - loss: 100133.7457 - mse: 100133.7422
Epoch 3/30
5000/5000 [==============================] - 0s 79us/sample - loss: 99354.4805 - mse: 99354.4766
Epoch 4/30
5000/5000 [==============================] - 0s 80us/sample - loss: 100002.0868 - mse: 100002.0703
Epoch 5/30
5000/5000 [==============================] - 0s 80us/sample - loss: 98612.6120 - mse: 98612.6172
Epoch 6/30
5000/5000 [==============================] - 0s 80us/sample - loss: 98367.5500 - mse: 98367.5234
Epoch 7/30
5000/5000 [==============================] - 0s 79us/sample - loss: 98449.4773 - mse: 98449.5078
Epoch 8/30
5000/5000 [==============================] - 0s 80us/sample - loss: 98179.5070 - mse: 98179.4922
Epoch 9/30
5000/5000 [==============================] - 0s 79us/sample - loss: 98145.2673 - mse: 98145.2812

In [188]:
print(y.shape)
print(X.shape)

(5000, 5)
(5000, 114, 5)


In [76]:
y = np.reshape(y,(-1,5))

In [77]:
y.shape

(5000, 5)

In [32]:
df = df.drop(['quote'],axis=1,inplace=True)

In [33]:
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [44]:
df1 = pd.read_csv(data_path / file_csv[0],)
df1.head()

,age,city,dist,gender,make,year,model,new_used,quote
0,61,Belleville,10726,m,audi,2017,e-tron,used,352.599790
1,21,Wood Buffalo,9401,m,toyota,2001,camry,used,1723.855245
2,83,Kelowna,9427,m,acura,2016,mdx,used,221.792349
3,28,St. John,21090,m,ford,2003,explorer,used,1723.855245
4,56,Sarnia,7099,m,audi,2018,e-tron,used,354.748787


In [45]:
df1 = df1.drop(['quote'], axis=1, inplace=False)

In [46]:
X = df1.to_numpy()

In [47]:
X

array([[61, 'Belleville', 10726, ..., 2017, 'e-tron', 'used'],
       [21, 'Wood Buffalo', 9401, ..., 2001, 'camry', 'used'],
       [83, 'Kelowna', 9427, ..., 2016, 'mdx', 'used'],
       ...,
       [43, 'Montreal', 2619, ..., 2003, 'corrola', 'used'],
       [37, 'Laval', 16457, ..., 2017, 'c', 'used'],
       [20, 'Fredericton', 14860, ..., 2015, 's', 'used']], dtype=object)

In [15]:
df1 = pd.get_dummies(df1,drop_first=True)

In [16]:
df1.head()

,age,dist,year,quote,city_Airdrie,city_Ajax,city_Aurora,city_Barrie,city_Belleville,city_Blainville,...,model_mirano,model_r8,model_rsx,model_s,model_sierra,model_terrain,model_x,model_x5,model_yukon,new_used_used
0,61,10726,2017,352.599790,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,21,9401,2001,1723.855245,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,83,9427,2016,221.792349,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,28,21090,2003,1723.855245,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,56,7099,2018,354.748787,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
X1 = df1.to_numpy()
X1.shape

(5000, 140)